In [2]:
import os
import random
import json
import albumentations as A
import cv2
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
from PIL import Image, ImageColor
from utils.dataset import load_mask, load_bboxes, load_classIdxMap, load_stafflines
from utils.metropolis_hastings import metropolis_hastings, binomial_distribution, propose_state, probability_matrix

DATASET_PATH = os.path.join("..", "..", "datasets", "generated")
PNG_PATH = os.path.join(DATASET_PATH, "png")
BBOX_PATH = os.path.join(DATASET_PATH, "bbox")

AUG_PATH = os.path.join(DATASET_PATH, "aug")
OUT_PATH = os.path.join(DATASET_PATH, "unet_aug")
OUT_TRAIN_PATH_X = os.path.join(OUT_PATH, "train", "x")
OUT_TRAIN_PATH_Y = os.path.join(OUT_PATH, "train", "y")
OUT_TEST_PATH_X = os.path.join(OUT_PATH, "test", "x")
OUT_TEST_PATH_Y = os.path.join(OUT_PATH, "test", "y")
OUT_VALID_PATH_X = os.path.join(OUT_PATH, "valid", "x")
OUT_VALID_PATH_Y = os.path.join(OUT_PATH, "valid", "y")

with open(os.path.join(DATASET_PATH, "classlist.json")) as file:
    classlist = json.load(file)
    colormap = {c['color']: c['id'] for c in classlist}
    colormap_inv = {c['id']: c['color'] for c in classlist}

In [3]:
def getSystemParams(systems, idx, eps=0):
    
    sys = systems[idx]
    x0 = max(0, sys["x"] - eps)
    y0 = max(0, sys["y"] - eps)
    x1 = x0 + sys["width"] + 2 * eps
    y1 = y0 + sys["height"] + 2 * eps
    factor = 256 / (sys["height"] + 2 * eps)
    
    def map_box_coords(x, y, w, h):
        x = int((x - x0) * factor)
        y = int((y - y0) * factor)
        w = int(w * factor)
        h = int(h * factor)
        return x, y, w, h
    
    def map_img(img, interpolation=cv2.INTER_NEAREST):
        crop = img[y0:y1, x0:x1].astype(np.uint8)
        cy = (y1 + y0) // 2
        res = cv2.resize(crop, (int((x1 - x0) * factor), 256), interpolation=interpolation)
        # res = img[cy - 128: cy + 128, x0:x1]
        return res
        
    return map_box_coords, map_img

# Generate dataset from augmented data

In [4]:
import pandas as pd
from Preprocessing.dataset import listSubFiles, groupPaths


paths = listSubFiles(AUG_PATH, depth=3)
augmentations = groupPaths(paths, depth=2, return_group_names=True)[0]
out_idx_train = 1
out_idx_test = 1
out_idx_valid = 1
out_idx = 1

# make output directories
os.makedirs(OUT_TRAIN_PATH_X, exist_ok=True)
os.makedirs(OUT_TRAIN_PATH_Y, exist_ok=True)
os.makedirs(OUT_TEST_PATH_X, exist_ok=True)
os.makedirs(OUT_TEST_PATH_Y, exist_ok=True)
os.makedirs(OUT_VALID_PATH_X, exist_ok=True)
os.makedirs(OUT_VALID_PATH_Y, exist_ok=True)

# store original image name, augmentation, filenameX, filenameY
mapping = []

for idx, aug in enumerate(augmentations):
    print(f"Processing augmentation ({idx}/{len(augmentations)}): {aug}, starting with index {out_idx_train}, {out_idx_test} (train, test)")
    for s_idx in tqdm(range(1, 26)):
        
        sample_idx = f"{s_idx:03d}"

        # for each sample:
        x_path = os.path.join(AUG_PATH, aug, "x", f"{sample_idx}.png")
        y_path = os.path.join(AUG_PATH, aug, "y", f"{sample_idx}.png")
        orig_img_x = cv2.imread(x_path)
        orig_img_y = cv2.imread(y_path)
        
        bboxes = load_bboxes(BBOX_PATH, sample_idx, img_shape=orig_img_x.shape)
        systems = {bbox["id"]: bbox for bbox in bboxes if bbox["type"]=="System"}
        unique_types = list(set([bbox["type"] for bbox in bboxes if bbox["type"] != "System"]))
        
        for idx, type in enumerate(unique_types):
            
            # get all bboxes of current type
            tbs = [bbox for bbox in bboxes if bbox["type"] == type]
            if len(tbs)==0:
                print("no box found for:", type)
                continue
            
            # pick random bbox
            rand_idx = random.randint(0, len(tbs) - 1)
            bbox = tbs[rand_idx]
            cx = bbox["cx"]
            cy = bbox["cy"]
            w = bbox["width"]
            h = bbox["height"]
            
            # map to system coordinates
            map_box_coords, map_img = getSystemParams(systems, int(bbox["systemId"]))
            cx, cy, w, h = map_box_coords(cx, cy, w, h)
            img_x = map_img(orig_img_x)
            
            # get frame centered around bbox center
            x0 = cx - 128
            x1 = cx + 128
            if x0 < 0:
                x0 = 0
                x1 = 256
            if x1 > img_x.shape[1]:
                x1 = img_x.shape[1]
                x0 = x1 - 256
                
            # split into train and test data
            if s_idx <= 4:
                out_path_x = OUT_VALID_PATH_X
                out_path_y = OUT_VALID_PATH_Y
                out_idx = out_idx_valid
                out_idx_valid += 1
            elif s_idx <= 20:
                out_path_x = OUT_TRAIN_PATH_X
                out_path_y = OUT_TRAIN_PATH_Y
                out_idx = out_idx_train
                out_idx_train += 1
            else:
                out_path_x = OUT_TEST_PATH_X
                out_path_y = OUT_TEST_PATH_Y
                out_idx = out_idx_test
                out_idx_test += 1
            
            # crop and save x
            img_x = img_x[:, x0:x1]
            x_out_path = os.path.join(out_path_x, f"{out_idx:04d}.png")
            cv2.imwrite(x_out_path, 255 - img_x)
            
            # crop and save y
            img_y = map_img(orig_img_y)[:, x0:x1]
            y_out_path = os.path.join(out_path_y, f"{out_idx:04d}.png")
            cv2.imwrite(y_out_path, img_y)
            
            mapping.append([x_path, y_path, aug, unique_types[idx], x_out_path, y_out_path])

df = pd.DataFrame(mapping)

Processing augmentation (0/10): dilation_all_k3, starting with index 1, 1 (train, test)


100%|██████████| 25/25 [00:21<00:00,  1.18it/s]


Processing augmentation (1/10): dilation_all_k5, starting with index 769, 241 (train, test)


100%|██████████| 25/25 [00:16<00:00,  1.47it/s]


Processing augmentation (2/10): dilation_stafflines_k3, starting with index 1537, 481 (train, test)


...

Processing augmentation (8/10): staffline_thickness_variation, starting with index 6145, 1921 (train, test)


100%|██████████| 25/25 [00:16<00:00,  1.49it/s]


Processing augmentation (9/10): staffline_y_variation, starting with index 6913, 2161 (train, test)


100%|██████████| 25/25 [00:16<00:00,  1.55it/s]


OSError: Cannot save file into a non-existent directory: 'plots'

In [6]:
OUT_SPLIT_FILE = os.path.join("..", "..", "datasets", "generated")
os.makedirs(OUT_SPLIT_FILE, exist_ok=True)
df.to_csv(os.path.join(OUT_SPLIT_FILE, "split.csv"), sep=';', header=False, index=False)